### Exercises (Chapter 4)

#### CRUD operations:


**1**: You will continue to work on the MCU network for a while. Who are the 10 fastest in the MCU ? Are they related to each other ?

**2**: How many have collaborated atleast more than 10 times with another superhero?

**3**: Did the weakest of the MCU collaborated with the strongest ?

**4**: Give a stats that interests you !

**5**: Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, Group of 3 that collab (undirected relation).

**6**: Get the age for people YOU collaborate with.

**7**: Get nodes with age > 25.

### Real world problem:

**8**: Using the downloaded data in TODO 11 of chapter II, create a collaboration network between authors for papers after 2021 (same author if same name). Insert it in neo4j in a db called "arxiv".

**9**: Get the density of degrees of the collaboration network

**10**: Do a for loop for i in range(500) for each i remove nodes with a degree less than i.


In [4]:
# 1: You will continue to work on the MCU network for a while. Who are the 10 fastest in the MCU ? Are they related to each other ?

import pandas as pd
import tqdm
import py2neo

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="mcu")


# Query for the 10 fastest superheroes
query_fastest_superheroes = """
MATCH (s:Superhero)
WHERE s.Speed IS NOT NULL
RETURN s.name AS Superhero, s.Speed AS Speed
ORDER BY s.Speed DESC
LIMIT 10
"""

result_fastest_superheroes = graph.run(query_fastest_superheroes).data()

# Print the result
print("Top 10 fastest superheroes:")
for superhero in result_fastest_superheroes:
    print(superhero["Superhero"], "-", superhero["Speed"])


Top 10 fastest superheroes:
Black Bolt - 100
Stardust - 100
Quicksilver - 100
Nova - 100
Thor - 92
Silver Surfer - 84
Galactus - 83
Dormammu - 83
Ego - 83
Northstar - 83


In [15]:
# 2: How many have collaborated atleast more than 10 times with another superhero?

query_collaborations = """
MATCH (a:Superhero)-[r]->(b:Superhero)
WITH a, COUNT(r) AS num_collaborations
WHERE num_collaborations >= 10
RETURN COUNT(a) AS num_superheroes
"""

result_num_superheroes = graph.run(query_collaborations).data()
num_superheroes = result_num_superheroes[0]["num_superheroes"]
print(f"Number of superheroes who have collaborated at least 10 times: {num_superheroes}")


Number of superheroes who have collaborated at least 10 times: 696


In [16]:
# 3: Did the weakest of the MCU collaborated with the strongest ?

query_weakest_strongest_collaboration = """
MATCH (weakest:Superhero), (strongest:Superhero)
WHERE weakest.Strength IS NOT NULL AND strongest.Strength IS NOT NULL
WITH MIN(weakest.Strength) AS min_strength, MAX(strongest.Strength) AS max_strength
MATCH (weakest)-[r]->(strongest)
RETURN COUNT(r) > 0 AS collaboration_exists
"""

result_collaboration_exists = graph.run(query_weakest_strongest_collaboration).data()
collaboration_exists = result_collaboration_exists[0]["collaboration_exists"]

if collaboration_exists:
    print("Yes, the weakest and strongest superheroes have collaborated.")
else:
    print("No, the weakest and strongest superheroes have not collaborated.")


Yes, the weakest and strongest superheroes have collaborated.


In [18]:
# 4: Give a stats that interests you ! 

# For example, let's calculate the average strength of all superheroes.

query_avg_strength = """
MATCH (s:Superhero)
WHERE s.Strength IS NOT NULL
RETURN AVG(s.Strength) AS AvgStrength
"""

avg_strength = graph.run(query_avg_strength).evaluate()
print("Average strength of superheroes:", avg_strength)


Average strength of superheroes: 35.55837563451778


In [20]:
# 5: Create a new db called "master". Create a graph from the M1 class with name and random age between 20 and 30, Group of 3 that collab (undirected relation).

import random  # Need to import the random module

# Create a new graph database called "master"
master_graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="master")

# Generate data for M1 class with random age between 20 and 30
# Assuming M1 class has properties 'name' and 'age'
m1_data = [
    {"name": "Person1", "age": random.randint(20, 30)},
    {"name": "Person2", "age": random.randint(20, 30)},
    {"name": "Person3", "age": random.randint(20, 30)}
]

# Create nodes and relationships in the master graph
for person in m1_data:
    master_graph.run("""
    CREATE (p:Person {name: $name, age: $age})
    """, name=person["name"], age=person["age"])

# Create relationships between groups of 3 people (undirected)
for i in range(0, len(m1_data), 3):
    master_graph.run("""
    MATCH (p1:Person {name: $name1}), (p2:Person {name: $name2}), (p3:Person {name: $name3})
    CREATE (p1)-[:COLLABORATES_WITH]->(p2),
           (p2)-[:COLLABORATES_WITH]->(p3),
           (p3)-[:COLLABORATES_WITH]->(p1)
    """, name1=m1_data[i]["name"], name2=m1_data[i + 1]["name"], name3=m1_data[i + 2]["name"])

In [23]:
# 6: Get the age for people YOU collaborate with.

# Assuming your name is stored in a variable named 'your_name'
your_name = "Person1"

query_collaborators_age = f"""
MATCH (you:Person {{name: '{your_name}'}})-[:COLLABORATES_WITH]-(collaborator)
RETURN collaborator.name AS Collaborator, collaborator.age AS Age
"""

collaborators_age = master_graph.run(query_collaborators_age).data()

# Print the result
print("Age of people you collaborate with:")
for collaborator in collaborators_age:
    print(collaborator["Collaborator"], "-", collaborator["Age"])

Age of people you collaborate with:
Person2 - 29
Person3 - 30


In [25]:
# 7: Get nodes with age > 25.

query_age_above_25 = """
MATCH (m:Person)
WHERE m.age > 25
RETURN m.name AS Name, m.age AS Age
"""

nodes_age_above_25 = master_graph.run(query_age_above_25).data()

# Print the result
print("Nodes with age above 25:")
for node in nodes_age_above_25:
    print(node["Name"], "-", node["Age"])

Nodes with age above 25:
Person1 - 30
Person2 - 29
Person3 - 30


In [ ]:
import pymongo
from py2neo import Graph, Node, Relationship
import json

# Connexion à Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password_tuts"), name="author")

# Ouvrir et lire le fichier CSV
with open("authors.csv", "r", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

# Créer des noeuds d'auteur dans Neo4j
for author in data["authors"]:
    author_node = Node("Author", name=author["name"])
    graph.merge(author_node)

    
    
#8: Using the downloaded data in TODO 11 of chapter II, create a collaboration network between authors for papers after 2021 (same author if same name). Insert it in neo4j in a db called "arxiv".

# Filtrer les articles publiés après 2021
query = {"PubYear": {"$gt": "2021"}}
papers = collection.find(query)

# Créer le réseau de collaboration entre les auteurs
for paper in papers:
    authors = paper.get("authors", [])
    for i in range(len(authors)):
        for j in range(i + 1, len(authors)):
            author1_name = authors[i].get("LastName")
            author2_name = authors[j].get("LastName")
            author1_node = Node("Author", name=author1_name)
            author2_node = Node("Author", name=author2_name)
            collaboration = Relationship(author1_node, "COLLABORATES_WITH", author2_node)
            graph.merge(collaboration)

            
#9: Get the density of degrees of the collaboration network

# Obtenir la densité des degrés du réseau de collaboration
query_density = """ 
MATCH (a:Author)-[:COLLABORATES_WITH]->(b:Author)
RETURN COUNT(*) / ((COUNT(DISTINCT a) * COUNT(DISTINCT b)) * 1.0) AS density
""" 
density = graph.run(query_density).evaluate()
print("Densité du réseau de collaboration:", density)


# 10: Do a for loop for i in range(500) for each i remove nodes with a degree less than i.

# Boucle pour supprimer les nœuds avec un degré inférieur à un certain seuil pendant 500 itérations
for i in range(500):
    query_delete = f"""
    MATCH (a:Author)
    WITH a, SIZE((a)-[:COLLABORATES_WITH]->()) AS degree
    WHERE degree < {i}
    DETACH DELETE a
    """
    graph.run(query_delete)